# Import packages

In [ ]:
# Import all relevant packages

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import copy
import math
import shapely
import random
from shapely.geometry import LineString, Point
from sympy import Point, Line, pi, sin
from skimage import util
from scipy.ndimage.measurements import label
from sklearn.cluster import KMeans, MiniBatchKMeans
from PIL import Image, ImageDraw
from matplotlib import cm
import pandas as pd

# Define all analysis functions

In [ ]:
# Define a LinkedList
class Node:
    def __init__(self, data):
       self.data = data
       self.next = None
class LinkedList:
    def __init__(self):
       self.headvalue = None
    def __init__(self, data):
       self.headvalue = Node(data)
       x1, y1, x2, y2 = data
       self.startCoord = (x1, y1)
       self.endCoord = (x2, y2)
    def append(self, data):
      if self.headvalue.next == None:
        self.tailvalue = Node(data)
        self.headvalue.next = self.tailvalue
      else:
        self.tailvalue.next = Node(data)
        self.tailvalue = self.tailvalue.next
      x1, y1, x2, y2 = data
      self.endCoord = (x2, y2)
    def listprint(self):
        printvalue = self.headvalue
        while printvalue is not None:
            print(printvalue.data)
            printvalue = printvalue.next
    def printToPicture(self, pic):
        printvalue = self.headvalue
        
        while printvalue is not None:
          x1, y1, x2, y2 = printvalue.data
          cv2.line(pic, (x1, y1), (x2,y2), (255,255,255), 3)
          cv2.circle(pic, (x1,y1), 5, (255, 255, 255), 5)
          cv2.circle(pic, (x2,y2), 5, (255, 255, 255), 5)
          printvalue = printvalue.next
    def printAng(self):
      Line1 = self.headvalue
      Line2 = self.headvalue.next
      while Line2 is not None:
        rad = angleBetweenLines([Line1.data],[Line2.data])
        #print("l1 {} l2 {} rad is {}".format(Line1.data, Line2.data, rad))
        Line1 = Line2
        Line2 = Line2.next
    def findBiggestAng(self):
      Line1 = self.headvalue
      Line2 = self.headvalue.next
      rad = 0
      saveLine = Line2.data
      while Line2 is not None:
        currentRad = angleBetweenLines([Line1.data],[Line2.data])
        if currentRad > rad:
          rad = currentRad
          saveLine = Line2.data
        Line1 = Line2
        Line2 = Line2.next
      return saveLine


# Define all functions required by LinkedLists

# Append new node to LinkedList
def appendList(startList, newList):
  x1, y1 = startList.endCoord
  x2, y2 = newList.startCoord
  startList.append([x1, y1, x2, y2])
  currentNode = newList.headvalue
  while (currentNode != None):
    startList.append(currentNode.data)
    currentNode = currentNode.next

def min(x, y):
  if x <= y:
    return x
  else:
    return y

def max(x, y):
  if x>=y:
    return x
  else:
    return y

# Find the shortest distnace between a point and the edge of the image
def findShortestPathToEdgePoint(x, y):
  minimumX = min(x, width - x)
  minimumY = min(y, height - y)
  minimumTotal = min(minimumX, minimumY)
  return minimumTotal

# Find the shortest distance between a line and the edge of the image
def findShortestPathToEdge(line):
  x1, y1, x2, y2 = line[0]
  firstPointMin = findShortestPathToEdgePoint(x1, y1)
  secondPointMin = findShortestPathToEdgePoint(x2, y2)
  AbsoluteMin = min(firstPointMin, secondPointMin)
  return AbsoluteMin

# Find the shortest distance between the `left' point of a segment and the 
# edge of the image
def findShortestPathToEdgeLeft(line):
  x1, y1, x2, y2 = line[0]
  PointMin = findShortestPathToEdgePoint(x1, y1)
  return PointMin

# Find the shortest distance between the `right' point of a segment and the 
# edge of the image
def findShortestPathToEdgeRight(line):
  x1, y1, x2, y2 = line[0]
  PointMin = findShortestPathToEdgePoint(x2, y2)
  return PointMin

# Find the distance between two points
def distanceBetweenPoints(x1, y1, x2, y2):
  run = abs(x2 - x1)
  rise = abs(y2 - y1)
  distance = (run * run + rise * rise) ** 0.5
  return distance

# Find the minimum distance between two lines
def findDistanceBetweenLines(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  minimum = distanceBetweenPoints(xStartA, yStartA, xStartB, yStartB)
  minimum = min(minimum,
                distanceBetweenPoints(xStartA, yStartA, xEndB, yEndB))
  minimum = min(minimum,
                distanceBetweenPoints(xEndA, yEndA, xStartB, yStartB))
  minimum = min(minimum,
                distanceBetweenPoints(xEndA, yEndA, xEndB, yEndB))
  return minimum

# Find the minimum distance between the `left' point of lineA and another line
def findDistanceToLinesLeft(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  minimum = distanceBetweenPoints(xStartA, yStartA, xStartB, yStartB)
  minimum = min(minimum,
                distanceBetweenPoints(xStartA, yStartA, xEndB, yEndB))
  return minimum

# Find the minimum distance between the `right' point of lineA and another line
def findDistanceToLinesRight(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  minimum = distanceBetweenPoints(xEndA, yEndA, xStartB, yStartB)
  minimum = min(minimum,
                distanceBetweenPoints(xEndA, yEndA, xEndB, yEndB))
  return minimum

# Find the angle between two lines
def angleBetweenLines(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  l1 = Line((xStartA, yStartA), (xEndA, yEndA))
  l2 = Line((xStartB, yStartB), (xEndB, yEndB))
  xTest, yTest = findMidpoint(xStartA, yStartA, xEndB, yEndB)
  directionCoef = 1
  if closed_image[yTest][xTest] >= 1: 
    directionCoef = -1
  rad = pi + (directionCoef * l1.angle_between(l2))
  return rad

# Connect two lines
# returns greatest distance between combination of the four coordinates
def connectLines(lineA, lineB): 
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  switchVariable = 1
  option1 = distanceBetweenPoints(xStartA, yStartA, xStartB, yStartB)
  option2 = distanceBetweenPoints(xStartA, yStartA, xEndB, yEndB)
  option3 = distanceBetweenPoints(xEndA, yEndA, xStartB, yStartB)
  option4 = distanceBetweenPoints(xEndA, yEndA, xEndB, yEndB)
  lineADist = distanceBetweenPoints(xStartA, yStartA, xEndA, yEndA)
  lineBDist = distanceBetweenPoints(xStartB, yStartB, xEndB, yEndB)
  max = copy.deepcopy(option1)
  if option2 > max:
    switchVariable += 1
    max = copy.deepcopy(option2)
  if option3 > max:
    switchVariable = 3
    max = copy.deepcopy(option3)
  if option4 > max:
    switchVariable = 4
    max = copy.deepcopy(option4)
  if lineADist > max:
    switchVariable = 5
    max = copy.deepcopy(lineADist)
  if lineBDist > max:
    switchVariable = 6
  if switchVariable is 1:
    return [[xStartA, yStartA, xStartB, yStartB]]
  elif switchVariable is 2:
    return [[xStartA, yStartA, xEndB, yEndB]]
  elif switchVariable is 3:
    return [[xEndA, yEndA, xStartB, yStartB]]
  elif switchVariable is 4:
    return [[xEndA, yEndA, xEndB, yEndB]]
  elif switchVariable is 5:
    return [[xStartA, yStartA, xEndA, yEndA]]
  else:
    return [[xStartB, yStartB, xEndB, yEndB]]

# does the exact same thing as connectLines
# thought same line would be an edge case but it turns out to be the same function
def connectLinesSameLine(lineA, lineB): 
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  switchVariable = 1
  option1 = distanceBetweenPoints(xStartA, yStartA, xStartB, yStartB)
  option2 = distanceBetweenPoints(xStartA, yStartA, xEndB, yEndB)
  option3 = distanceBetweenPoints(xEndA, yEndA, xStartB, yStartB)
  option4 = distanceBetweenPoints(xEndA, yEndA, xEndB, yEndB)
  lineADist = distanceBetweenPoints(xStartA, yStartA, xEndA, yEndA)
  lineBDist = distanceBetweenPoints(xStartB, yStartB, xEndB, yEndB)
  max = copy.deepcopy(option1)
  if option2 > max:
    switchVariable += 1
    max = copy.deepcopy(option2)
  if option3 > max:
    switchVariable = 3
    max = copy.deepcopy(option3)
  if option4 > max:
    switchVariable = 4
    max = copy.deepcopy(option4)
  if lineADist > max:
    switchVariable = 5
    max = copy.deepcopy(lineADist)
  if lineBDist > max:
    switchVariable = 6
  if switchVariable == 1:
    return [[xStartA, yStartA, xStartB, yStartB]]
  elif switchVariable == 2:
    return [[xStartA, yStartA, xEndB, yEndB]]
  elif switchVariable == 3:
    return [[xEndA, yEndA, xStartB, yStartB]]
  elif switchVariable == 4:
    return [[xEndA, yEndA, xEndB, yEndB]]
  elif switchVariable == 5:
    return [[xStartA, yStartA, xEndA, yEndA]]
  else:
    return [[xStartB, yStartB, xEndB, yEndB]]


# connect lines that are close and of a similar angle
def smoothOutLines():
  # print("smoothOutLines")
  i = 0
  while i < len(lines): # the same line may connect with multiple nearby lines
    if lines[i] is NullLine: # skips
      i+= 1
      continue
    minimumLeft = findShortestPathToEdgeLeft(lines[i]) # start point
    minimumRight = findShortestPathToEdgeRight(lines[i]) # end point
    closestLineIndexLeft = i
    closestLineIndexRight = i
    # print("minimumLeft is {} minimum right is {}".format(minimumLeft, minimumRight))
    # two for loops inefficent. 
    # Is it possible to use some sort of clustering method? 
    # To go through once and sort closest lines left to right
    for j in range(0, len(lines)): # finds closest line to the left
      if i is not j and lines[j] is not NullLine:
        distanceToLinesLeft = findDistanceToLinesLeft(lines[i], lines[j])
        if distanceToLinesLeft < minimumLeft:
          closestLineIndexLeft = j
          minimumLeft = distanceToLinesLeft
    for j in range(0, len(lines)): # finds closest line to the right
      if i is not j and lines[j] is not NullLine and j is not closestLineIndexLeft:
        distanceToLinesRight = findDistanceToLinesRight(lines[i], lines[j])
        if distanceToLinesRight < minimumRight:
          closestLineIndexRight = j 
          minimumRight = distanceToLinesRight
    merge = False # if there is no merge, this line cannot merge, move to next. 
                  # Else, continue working on this line
    # print("i is {} jleft is {} jright is {}".format(i, closestLineIndexLeft, closestLineIndexRight))
    # print("line is {} left line is {} right line is {}".format(lines[i], lines[closestLineIndexLeft], lines[closestLineIndexRight]))
    if closestLineIndexLeft is closestLineIndexRight and lines[closestLineIndexLeft][0][0] >=0 and closestLineIndexLeft is not i: #edge cases where left is right
      # print("connect same line")
      lines[i] = connectLinesSameLine(lines[closestLineIndexLeft], lines[closestLineIndexRight])
      lines[closestLineIndexLeft] = NullLine 
      merge = True
    else:
      if i is not closestLineIndexLeft and lines[closestLineIndexLeft][0][0] >= 0:
        rad = angleBetweenLines(lines[i], lines[closestLineIndexLeft])
        # print("rad left is {}".format(rad))
        if (rad >0 and rad < ANG) or (rad > pi - ANG and rad < pi + ANG) or (rad > 2 * pi - ANG):
          # print("merging left")
          testLine = connectLines(lines[i], lines[closestLineIndexLeft])
          rad = angleBetweenLines(lines[i], testLine)
          if (rad >0 and rad < ANG) or (rad > pi - ANG and rad < pi + ANG) or (rad > 2 * pi - ANG): #if angle is within range, merge
            lines[i] = testLine
            lines[closestLineIndexLeft] = NullLine # null lines will be deleted later
            # print("after left merge, line is {}".format(lines[i]))
            merge = True
      if i is not closestLineIndexRight and lines[closestLineIndexRight][0][0] >=0:
        rad = angleBetweenLines(lines[i], lines[closestLineIndexRight])
        # print("rad right is {}".format(rad))
        if (rad >0 and rad < ANG) or (rad > pi - ANG and rad < pi + ANG) or (rad > 2 * pi - ANG):
          # print("merging right")
          testLine = connectLines(lines[i], lines[closestLineIndexRight])
          rad = angleBetweenLines(lines[i], testLine)
          # print("rad {} line {} testline {}".format(rad, lines[i], testLine))
          if (rad >0 and rad < ANG) or (rad > pi - ANG and rad < pi + ANG) or (rad > 2 * pi - ANG):
            lines[i] = testLine
            lines[closestLineIndexRight] = NullLine
            # print("after right merge, line is {}".format(lines[i]))
            merge = True
    if merge is False:
      i += 1

# remove unused lines (null lines)    
def deleteNullLines(lines):
  newLines = []
  for points in lines:
    x1, y1, x2, y2 = points[0]
    if x1 >= 0:
      newLines.append(points)
  return newLines

# find the slope between two lines
def findSlope(x1, y1, x2, y2):
  rise = y2 - y1
  run = x2 - x1
  return run, rise
# find the smallest of 4 distance values
def whichOneIsSmallest(A1B1, A1B2, A2B1, A2B2):
  smallestIndex = 1
  smallestValue = A1B1
  if (A1B1 > A1B2):
    smallestIndex = 2
    smallestValue = A1B2
  if (smallestValue > A2B1):
    smallestIndex = 3
    smallestValue = A2B1
  if (smallestValue > A2B2):
    smallestIndex = 4
  return smallestIndex
  
# find the closest pair of points between two lines
def findClosestPoints(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA
  xStartB, yStartB, xEndB, yEndB = lineB
  A1B1 = distanceBetweenPoints(xStartA, yStartA, xStartB, yStartB)
  A1B2 = distanceBetweenPoints(xStartA, yStartA, xEndB, yEndB)
  A2B1 = distanceBetweenPoints(xEndA, yEndA, xStartB, yStartB)
  A2B2 = distanceBetweenPoints(xEndA, yEndA, xEndB, yEndB)
  option = whichOneIsSmallest(A1B1, A1B2, A2B1, A2B2)
  if option == 1:
    return xStartA, yStartA, xStartB, yStartB
  elif option == 2:
    return xStartA, yStartA, xEndB, yEndB
  elif option == 3:
    return xEndA, yEndA, xStartB, yStartB
  else:
    return xEndA, yEndA, xEndB, yEndB

# extend a line upto a new point
def extendLineToPoint(lineA, lineB, newX, newY):
  xStartA, yStartA, xEndA, yEndA = lineA
  xStartB, yStartB, xEndB, yEndB = lineB
  startDistanceA = distanceBetweenPoints(xStartA, yStartA, newX, newY)
  endDistanceA = distanceBetweenPoints(xEndA, yEndA, newX, newY)
  startDistanceB = distanceBetweenPoints(xStartB, yStartB, newX, newY)
  endDistanceB = distanceBetweenPoints(xEndB, yEndB, newX, newY)
  distanceA = distanceBetweenPoints(xStartA, yStartA, xEndA, yEndA)
  distanceB = distanceBetweenPoints(xStartB, yStartB, xEndB, yEndB)
  closestAx, closestAy, closestBx, closestBy = findClosestPoints(lineA, lineB)
  maxA = max(startDistanceA, endDistanceA)
  maxB = max(startDistanceB, endDistanceB)
  if maxA == distanceBetweenPoints(closestAx, closestAy, newX, newY):
    #print("MaxA")
    return lineA, lineB
  if maxB == distanceBetweenPoints(closestBx, closestBy, newX, newY):
    #print("MaxB")
    return lineA, lineB
  if startDistanceA <= endDistanceA:
    tempA = [newX, newY, xEndA, yEndA]
  else:
    tempA = [xStartA, yStartA, newX, newY]
  if startDistanceB <= endDistanceB:
    tempB = [newX, newY, xEndB, yEndB]
  else:
    tempB = [xStartB, yStartB, newX, newY]
  tax1, tay1, tax2, tay2 = tempA
  tbx1, tby1, tbx2, tby2 = tempB
  distanceTempA = distanceBetweenPoints(tax1, tay1, tax2, tay2)
  distanceTempB = distanceBetweenPoints(tbx1, tby1, tbx2, tby2)
  # print("tempA {} tempB {}".format(tempA, tempB))
  if (tax1 == tbx1 and tay1 == tby1) or (tax2 == tbx2 and tay2 == tby2): #handles edge case that can screw the whole algorithm up
    # print("samdirection")
    return lineA, lineB
  if (distanceTempA > 2 * distanceA or distanceTempB > 2 * distanceB):
    # print("toolong")
    return lineA, lineB
  return tempA, tempB

# find the intercept of a line
def findIntercept(x, y, dx, dy):
  slope = dy/dx
  intercept = y - (x * slope)
  return intercept

# extend a line - deals with edge case where lineB is vertical
def findNewXNewYIfVertical(Line, verticalLine, ux, uy):
  xStartA, yStartA, xEndA, yEndA = Line
  xStartB, yStartB, xEndB, yEndB = verticalLine
  xDifference = xStartB - xStartA
  slope = uy/ux
  newY = yStartA + (xDifference * slope)
  return int(round(xStartB)), int(round(newY))
  
# merge the `extreme' vertices of two lines
def mergeVertices(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA
  xStartB, yStartB, xEndB, yEndB = lineB
  ux, uy = findSlope(xStartA, yStartA, xEndA, yEndA) #slope in x and y direction for one line
  vx, vy = findSlope(xStartB, yStartB, xEndB, yEndB) #for second line
  b1 = findIntercept(xStartA, yStartA, ux, uy) #intercept for line 1
  b2 = findIntercept(xStartB, yStartB, vx, vy) #intercept for line 2
  #t2 is the scalar used to figure out how much I need to multiply to find intersection point
  #this involved a lot of algebra and vector calc. 
  t2 = (yStartA + xStartB*uy/ux - xStartA *uy/ux - yStartB) / (vy - vx * uy/ux)
  #print("t2 {} lineA {} lineB {} b1 {} b2 {} ux {} uy {} vx {} vy {}".format(t2, lineA, lineB, b1, b2, ux, uy, vx, vy))
  if ux == 0 and vx == 0:
    return lineA, lineB #will throw an error, abort operation
  elif ux == 0:
    newx, newy = findNewXNewYIfVertical(lineB, lineA, vx,vy) #will throw an error if using next function, use this edge case function
  elif vx == 0:
    newx, newy = findNewXNewYIfVertical(lineA, lineB, ux,uy)
  elif vy / vx == uy / ux: #parallel lines
    return lineA, lineB
  else:
    #print("lineA {} lineB {} t2 {} vx {} vy {}".format(lineA, lineB, t2, vx, vy))
    newx = int(round(xStartB + t2 * vx))
    newy = int(round(yStartB + t2 * vy))
  if newx >= 0 and newx <= width and newy >=0 and newy <= height:
    lineA, lineB = extendLineToPoint(lineA, lineB, newx, newy)
    #lineA = extendLineToPoint(lineA, newx, newy)
    #lineB = extendLineToPoint(lineB, newx, newy)
  return lineA, lineB

# connect vertices. Uses slope of the line to figure out where they intersect
# works by extending line through slope to find where they intersect. 
# changes closest coordinates of both lines to intersection point
def connectVertices(lines):
  # print("connectVertices")
  i = 0
  while i < len(lines): # similar algorithm to smoothlines
    minimumLeft = findShortestPathToEdgeLeft(lines[i])
    minimumRight = findShortestPathToEdgeRight(lines[i])
    closestLineIndexLeft = i # copy.deepcopy(i)
    closestLineIndexRight = i
    for j in range(0, len(lines)):
      if i is not j and lines[j] is not NullLine:
        distanceToLinesLeft = findDistanceToLinesLeft(lines[i], lines[j])
        distanceToLinesRight = findDistanceToLinesRight(lines[i], lines[j])
        if distanceToLinesLeft < minimumLeft:
          closestLineIndexLeft = j # copy.deepcopy(j)
          minimumLeft = distanceToLinesLeft
        if distanceToLinesRight < minimumRight:
          closestLineIndexRight = j # copy.deepcopy(j)
          minimumRight = distanceToLinesRight
    # print("i {} left {} right {}".format(lines[i][0], ...
    # ... lines[closestLineIndexLeft][0], lines[closestLineIndexRight][0]))
    if i != closestLineIndexLeft: # and closestLineIndexLeft != closestLineIndexRight:
      # print("left merge")
      lines[i][0],lines[closestLineIndexLeft][0] = mergeVertices(lines[i][0], lines[closestLineIndexLeft][0])
      # print("after left merge line {} left {}".format(lines[i][0],lines[closestLineIndexLeft][0]))
    if i != closestLineIndexRight:# and closestLineIndexLeft != closestLineIndexRight:
      # print("right merge")
      lines[i][0],lines[closestLineIndexRight][0] = mergeVertices(lines[i][0], lines[closestLineIndexRight][0])
      # print("after right merge line {} right {}".format(lines[i][0],lines[closestLineIndexRight][0]))
    i += 1

def findMidpoint(x1, y1, x2, y2):
  xMid = (x2 + x1) / 2
  yMid = (y1 + y2) / 2
  return int(round(xMid)), int(round(yMid))

def isCorrectOrientation(x, y, dx, dy):
  distance = .001 # basically finds the difference between left and right side. May cause error if close to edge
  while closed_image[int(round(y + dx * distance))][int(round(x - dy * distance))] == closed_image[int(round(y - dx * distance))][int(round(x + dy * distance))]:
    distance += .001
  if closed_image[int(round(y - dx * distance))][int(round(x + dy * distance))] > closed_image[int(round(y + dx * distance))][int(round(x - dy * distance))]:
    return True
  return False

def reOrientLine(line): # counter clockwise orientation
  x1, y1, x2, y2 = line[0]
  x3, y3 = findMidpoint(x1, y1, x2, y2)
  dx, dy = findSlope(x1, y1, x2, y2)
  if isCorrectOrientation(x3, y3, dx, dy) is False:
    line[0] = x2, y2, x1, y1
  return line

def reOrientLines(lines):
  # print("reorientLines")
  for line in lines:
    line = reOrientLine(line)


def findBeginningCoordinates(lines):
  x1, y1, x2, y2 = lines[0][0]
  startPoints = {(x1, y1)}
  endPoints = {(x2, y2)}
  for i in range(1, len(lines)):
    x1, y1, x2, y2 = lines[i][0]
    startPoints.add((x1, y1))
    endPoints.add((x2, y2))
  return startPoints, endPoints

def createSegment(line, lines, endCoord, startPoints):
  # print("CreateSegment")
  # print("line {} endCoord {} startPoints {}".format(line, endCoord, startPoints))
  x1, y1, x2, y2 = line
  startPoints.remove((x1, y1)) # don't need it anymore
  LList = LinkedList(line)
  while((x2, y2) not in endCoord): # go through list until you hit end coordinate
    for i in range(0, len(lines)):
      Bx1, By1, Bx2, By2 = lines[i][0]
      if x2 == Bx1 and y2 == By1:
        LList.append(lines[i][0])
        x1, y1, x2, y2 = lines[i][0]
        startPoints.remove((x1, y1))
  return LList


def findNewStart(LList):
  startLine = LList.findBiggestAng()
  newLList = LinkedList(startLine)
  currentNode = LList.headvalue
  passedFlag = False
  while currentNode is not None:
    if passedFlag is True:
      newLList.append(currentNode.data)
    elif currentNode.data is startLine:
      passedFlag = True
    currentNode = currentNode.next
  currentNode = LList.headvalue
  while currentNode.data is not startLine:
    newLList.append(currentNode.data)
    currentNode = currentNode.next
  return newLList
    

def findCircularSegments(lines, startPoints, segments):
  #print("THIS FUNCTION IS CALLED")
  for line in lines:
    x1, y1, x2, y2 = line[0]
    if (x1, y1) in startPoints: #find arbitrary start
      xNext = x2
      yNext = y2
      xFinal = x1
      yFinal = y1
      startPoints.remove((x1, y1))
      LList = LinkedList(line[0])
      #print("xFinal {} yFinal {} xNext {} yNext {}".format(xFinal, yFinal, xNext, yNext))
      break
  if len(startPoints) == 0:
    return
  while (xNext != xFinal or yNext != yFinal): #continue until end equals original start
    #print("while startpoints{}".format(startPoints))
    for line in lines:
      x1, y1, x2, y2 = line[0]
      if x1 == xNext and y1 == yNext:
        #print("line {}".format(line[0]))
        LList.append(line[0])
        startPoints.remove((x1, y1))
        xNext = x2
        yNext = y2
  LList = findNewStart(LList) #recreate linked list with start that makes more sense
  segments.append(LList)
      

#organize lines into linked list
def findSegments(lines):
  segments = []
  startPoints, endPoints = findBeginningCoordinates(lines) #sets of start and end points for each line
  beginningCoord = startPoints - endPoints #beginning of linked lists
  endCoord = endPoints - startPoints #end of linked lists
  for line in lines:
    x1, y1, x2, y2 = line[0]
    if (x1, y1) in beginningCoord:
      LList = createSegment(line[0], lines, endCoord, startPoints)
      segments.append(LList)
  while len(startPoints) > 0: #circular segments will have no start nor end point
    findCircularSegments(lines, startPoints, segments)
  return segments

def findArea(lines):
  hexagonArea = 0
  for line in lines:
    x1, y1, x2, y2 = line
    hexagonArea += (x2 + x1) * (y2 - y1) / 2
  return abs(hexagonArea)

def findMultiplier(rad):
  if rad < pi/6:
    return 0
  elif rad < 2 * pi / 6:
    return 10.0 # 10.0
  elif rad < 3 * pi / 6:
    return 6.0 # 6.0
  elif rad < 4 * pi / 6:
    return 4.0 # 4.0
  elif rad < 5 * pi / 6:
    return 3.0 # 3.0
  elif rad < 6 * pi / 6:
    return 2.0 # 2.0
  elif rad < 7 * pi / 6:
    return 1.2 # 1.2
  else:
    return 1.0
  # else:
  #   return float((0.75*2*pi)/(rad-sin(rad)))

def totalAngleBetweenLines(lines): 
  rad = 0
  for i in range(1, len(lines)):
    xStartA, yStartA, xEndA, yEndA = lines[i-1]
    xStartB, yStartB, xEndB, yEndB = lines[i]
    l1 = Line((xStartA, yStartA), (xEndA, yEndA))
    l2 = Line((xStartB, yStartB), (xEndB, yEndB))
    rad += l1.angle_between(l2)
  return rad


def addHexagonArea(lines, area, numHexagons):
  if len(lines) == 1:
    return [], area, numHexagons
  xStartA, yStartA, xEndA, yEndA = lines[0]
  xStartB, yStartB, xEndB, yEndB = lines[len(lines) - 1]
  numHexagons += 1
  if xStartA == xEndB and yStartA == yEndB:
    newArea = findArea(lines)
    if newArea > 5000: #ignore tiny hexagons
      area += newArea
    else:
      numHexagons -= 1
    return [], area, numHexagons
  rad = totalAngleBetweenLines(lines)
  #print("rad is {} lines {}".format(rad, lines))
  multiplier = findMultiplier(rad)
  #if multiplier == 0:
    #numHexagons -= 1
  lines.append([xEndB, yEndB, xStartA, yStartA])
  newArea = (multiplier * findArea(lines))
  if newArea > 5000:
    area += newArea
  else:
    numHexagons -= 1
  return [], area, numHexagons

def findOuterHexagonArea(segment, area, numHexagons):
  #print("findouterarea")
  currentNode = segment.headvalue
  if currentNode.next is None: #ignores segments of length 1
    return area, numHexagons
  lines = []
  while currentNode.next is not None:
    lines.append(currentNode.data)
    x1, y1, x2, y2 = lines[0]
    Bx1, By1, Bx2, By2 = currentNode.data
    temprad = angleBetweenLines([currentNode.data], [currentNode.next.data])
    #print("rad {} lines {}".format(temprad, lines))
    if angleBetweenLines([currentNode.data], [currentNode.next.data]) > pi + (2* ANG): #2 ang is bias
      #print("lines {}".format(lines))
      lines, area, numHexagons = addHexagonArea(lines, area, numHexagons)
      #print("area {} num {}".format(area, numHexagons))
    if len(lines) > 1 and Bx2 == x1 and By2 == y1: #circular hexagon
      lines, area, numHexagons = addHexagonArea(lines, area, numHexagons)
    currentNode = currentNode.next
  lines.append(currentNode.data)
  #print("lines {}".format(lines))
  lines, area, numHexagons = addHexagonArea(lines, area, numHexagons) #add final hexagon area
  #print("area {} num {}".format(area, numHexagons))
  return area, numHexagons



def connectSegments(segments): #connects two close segements
  counter = 0
  while (counter < len(segments)):
    xEnd, yEnd = segments[counter].endCoord
    endMinimum = findShortestPathToEdgePoint(xEnd, yEnd)
    segmentIndex = counter
    for j in range (0, len(segments)):
      if j != counter:
        xBegin, yBegin = segments[j].startCoord
        distance = distanceBetweenPoints(xEnd, yEnd, xBegin, yBegin)
        if endMinimum > distance:
          segmentIndex = j
          endMinimum = distance
    if segmentIndex != counter:
      appendList(segments[counter], segments[segmentIndex])
      segments.remove(segments[segmentIndex])
    counter += 1

# Analysis of randomly generated data

In [ ]:
# Define all variables: 

# mean and std of gaussian for number of polygons to insert in the image
mean_number_of_polygons = 10
std_number_of_polygons = 1

# mean and std of gaussian from which we pick the side length
mean_side_length = 100
std_side_length = 30

polygon_stretch = 20 # pick value from 0 to 100; higher value means lesser regular polygon
cutoff_area = 5000 # cutoff for hexagons that are too small

number_of_images = 2 # number of images to test the code on
image_size = 1024 # length of side of image (in pixels)
canvas_stretch = 2*(mean_side_length+2*std_side_length) # number of pixels to extend image by
cropping = True # boolean switch to get cropped edges

add_artefacts = True # boolean switch to add artefacts to the image
artefact_size = 0.2 # radius of artefact as a percent of the mean side length
number_of_artefacts = 3 # number of artefacts as a percent of the number of polygons

# probabilities of having a particular number of side
side_list = [4, 5, 6, 7, 8, 9, 10]
probabilities = [0.02, 0.15, 0.5, 0.15, 0.10, 0.05, 0.03]

# kernel for erosion and dilation
kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)

structure = np.ones((3, 3), dtype=int)

In [ ]:
def point_in_circle(x_in, y_in, radius_in, scale):
  ''' Gives a point which lies within a circle of radius `radius_in` 
  and centre `(x_in, y_in)`'''
  # random angle
  alpha = 2 * np.pi * random.random()
  # random radius
  r = radius_in * np.sqrt(random.random()) * scale/100
  # calculating coordinates
  x_out = r * np.cos(alpha) + x_in
  y_out = r * np.sin(alpha) + y_in

  return(x_out, y_out)

In [ ]:
def generate_image():
  # get number of polygons in the image from a normal distribution
  number_of_polygons = int(np.random.normal(mean_number_of_polygons, std_number_of_polygons))
  # list of number of sides of each polygon
  polygons_list = np.random.choice(side_list, number_of_polygons, p=probabilities)
  # generate the canvas
  img = np.zeros((image_size+canvas_stretch, image_size+canvas_stretch))
  # variables for future use
  old_area = 0
  true_number_of_hexagons = 0

  # loop through each polygon:
  for number_of_sides in polygons_list:
    # location of centroid
    centroid = [random.randint(canvas_stretch/2, img.shape[0]-canvas_stretch/2), 
                random.randint(canvas_stretch/2, img.shape[1]-canvas_stretch/2)]
    # internal angle of each side
    internal_angle = 360/number_of_sides 
    # orientation of the first side
    orientation = random.randint(0, int(internal_angle)-1)
    # side length of regular polygon; also radius of circle where polygon is inscribed
    side_length = np.random.normal(mean_side_length, std_side_length)
    # initialise a list of the x and y coordinates of the vertices
    vertices = []
    
    # loop through each side:
    for side in range(number_of_sides):
      # get x and y coordinates of vertex
      ideal_x = np.cos((orientation+side*internal_angle)*np.pi/180)*side_length + centroid[0]
      ideal_y = np.sin((orientation+side*internal_angle)*np.pi/180)*side_length + centroid[1]
      # add noise to vertex coordinates to make irregular polygons
      vertex_x, vertex_y = point_in_circle(ideal_x, ideal_y, side_length/2, polygon_stretch)
      # append the noisy coordinates to the list
      vertices.append([int(vertex_x), int(vertex_y)])
      # leave loop through each side
    
    # make a 2D numpy array of the x,y coordinates of the vertices
    vertices = np.array(vertices)
    # add the polygon to the canvas
    polygon = cv2.fillConvexPoly(img, points=vertices, color=255)
    # calculate the extra area that the polygon adds
    added_area = np.sum(polygon)/255 - old_area
    # if that area is greater than a cutoff value, add 1 to the true number of hexagons
    # this is done to deal with almost fully overlapping hexagons and small hexagons
    if added_area >= cutoff_area:
      true_number_of_hexagons += 1
    # update the value of old area
    old_area = np.sum(polygon)/255
    # leave loop through each polygon

  # if we do not want cropped hexagons in the image
  if not cropping:
    out_hexagon_number = true_number_of_hexagons
  # if we want cropped hexagons in the image
  else:
    out_hexagon_number = (np.sum(img[canvas_stretch//2:-canvas_stretch//2, 
                                     canvas_stretch//2:-canvas_stretch//2]))/\
                                     (np.sum(img)/true_number_of_hexagons)
  
  # if we want to add artefacts
  if add_artefacts:
    for artefact in range(number_of_artefacts*number_of_polygons):
      centre = (random.randint(canvas_stretch/2, img.shape[0]-canvas_stretch/2), 
                random.randint(canvas_stretch/2, img.shape[1]-canvas_stretch/2))
      radius = int(artefact_size*mean_side_length)
      img = cv2.circle(img, centre, radius, color=0, thickness=-1)
  # if we do not want cropped hexagons in the image
  if not cropping:
    out_img = np.uint8(img)
  # if we want cropped hexagons in the image
  else:
    out_img = np.uint8(img[canvas_stretch//2:-canvas_stretch//2, canvas_stretch//2:-canvas_stretch//2])

  return out_img, out_hexagon_number, true_number_of_hexagons

In [ ]:
# initialise the results list. This will be a list of dictionaries.
results_list = []

# loop through each image:
for image_number in range(number_of_images):
  # generate image and store its information
  image, areal_value, given_value = generate_image()
  # add the original image to the results list as an entry with blob number = 0
  results_list.append({'name':'Areal value: ' + str(round(areal_value, 3)) + '; Given value: '+str(given_value),
                       'array':image, 
                       'areal_value': areal_value,
                       'given_value': given_value, 
                       'image': image_number+1,
                       'blob': 0})
  # perform erosion and dilation to remove any super small domains
  erosion = cv2.erode(image, kernel, iterations=5)
  dilation = cv2.dilate(erosion, kernel, iterations=5)
  # label each connected blob
  labeled, ncomponents = label(dilation, structure)

  # loop through each blob:
  for blob_number in range(1, np.max(labeled)+1):
    # make sure the blob size is bigger than a threshold. Else the blob is too small and noisy
    if np.sum(np.where(labeled==blob_number, 1, 0))>cutoff_area:
      # get an image with just the current blob
      filtered_image = np.where(labeled==blob_number, 1, 0)
      filtered_image = filtered_image.astype('uint8')
      # copy that image
      res = filtered_image.copy()
      # fill any holes in that image
      contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
      for cnt in contour:
        cv2.drawContours(res,[cnt],0,255,-1)
      # copy the closed image
      closed_image = res.copy()

      ### ---- ANALYSIS CODE BELOW ---- 

      edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

      width = len(edges[0])
      height = len(edges)
      ANG = pi/12
      NullLine = [[-1, -1, -1, -1]]

      lines = cv2.HoughLinesP(image=edges, rho=1, theta=1*np.pi/180, threshold=25, minLineLength=5, maxLineGap=50)
      smoothOutLines()
      newLines = deleteNullLines(lines)
      reOrientLines(newLines)
      connectVertices(newLines)

      segments = findSegments(newLines)
      # connectSegments(segments)

      area = 0
      numHexagons = 0

      for segment in segments:
        area, numHexagons = findOuterHexagonArea(segment, area, numHexagons)

      for segment in segments:
        segment.printToPicture(edges)
# segment.printAng()

# for line in newLines:
#   x1, y1, x2, y2 = line[0]
#   print("x1 {} y1 {} x2 {} y2 {}".format(x1, y1, x2, y2))
      averageArea = area/numHexagons
      estimation = np.sum(closed_image) / (averageArea * 255)


      ### ---- ANALYSIS CODE ABOVE ---- 

      # append the analysed blob image to the results list
      results_list.append({'name':'blob number {0}; {1} hexagons'.format(blob_number, round(estimation, 3)), # change variable name with whatever your estimation value variable is
                           'array':edges, # Change `edges` with whatever your output variable name is
                           'areal_value': estimation, # change variable name with whatever your estimation value variable is
                           'given_value': int(round(estimation, 0)), # change variable name with whatever your estimation value variable is
                           'image': image_number+1,
                           'blob': blob_number})

In [ ]:
# ------ RUN THIS CELL ONLY IF YOU WANT TO DISPLAY THE IMAGE RESULTS ------ #

numberOfImages = len(results_list)

columns = 4 # you can change this based on what feels most comfortable for your viewing
fig, axes = plt.subplots(numberOfImages//columns+1, columns, figsize=(16, 16*(numberOfImages//columns+1)/columns), sharex=False, sharey=False)
for i, ax in enumerate(axes.flatten()):
  if i<numberOfImages:
    ax.imshow(results_list[i]['array'])
    ax.title.set_text(results_list[i]['name'])
    ax.set_axis_off()
plt.tight_layout()

In [ ]:
# results_list[-1]['image']
df = pd.DataFrame(index=range(1, results_list[-1]['image']+1),columns=['Areal value', 'Given value' ,'Estimated value'])
estimated_value = 0
image_number = 1
for result in results_list:
  print(result['image'], result['blob'], df.loc[result['image']]['Areal value'])
  # while result['image']==image_number:
  if result['blob']==0 and result['image']!=image_number:
    df.loc[result['image']-1]['Estimated value'] = estimated_value
    estimated_value = 0
    image_number = result['image']
    df.loc[result['image']]['Areal value'] = result['areal_value']
    df.loc[result['image']]['Given value'] = result['given_value']
  elif result['blob']==0 and result['image']==image_number:
    df.loc[result['image']]['Areal value'] = result['areal_value']
    df.loc[result['image']]['Given value'] = result['given_value']
  else:
    estimated_value += result['areal_value']
df.loc[result['image']]['Estimated value'] = estimated_value
# df

In [ ]:
df

In [ ]:
fig, ax = plt.subplots()
df.plot(y='Areal value', ax=ax)
df.plot(y='Given value', ax=ax)
df.plot(y='Estimated value', ax=ax)

## Follow process for one simulated image

In [ ]:
image2, areal_value2, given_value2 = generate_image()

plt.figure(figsize=(8,8))
plt.imshow(image2, cmap='gray')
plt.axis('off')

In [ ]:
image, areal_value, given_value = generate_image()

plt.figure(figsize=(8,8))
plt.imshow(image, cmap='gray')
plt.axis('off')

In [ ]:
image = cv2.imread('combined_data/20221115_2_F2_5000_mask.png', 0)

plt.figure(figsize=(8,8))
plt.imshow(image, cmap='gray')
plt.axis('off')

In [ ]:
# perform erosion and dilation to remove any super small domains
erosion = cv2.erode(image, kernel, iterations=5)
dilation = cv2.dilate(erosion, kernel, iterations=5)
# label each connected blob
labeled, ncomponents = label(dilation, structure)

plt.figure(figsize=(8,8))
plt.imshow(labeled, cmap='gray')
plt.axis('off')

In [ ]:
blob_number=5
filtered_image = np.where(labeled==blob_number, 1, 0)
filtered_image = filtered_image.astype('uint8')
# copy that image
res = filtered_image.copy()
# fill any holes in that image
contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
for cnt in contour:
  cv2.drawContours(res,[cnt],0,255,-1)
# copy the closed image
closed_image = res.copy()

plt.figure(figsize=(8,8))
plt.imshow(closed_image, cmap='gray')
plt.axis('off')

In [ ]:
edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

plt.figure(figsize=(8,8))
plt.imshow(edges, cmap='gray')
plt.axis('off')

In [ ]:
width = len(edges[0])
height = len(edges)
ANG = pi/12
NullLine = [[-1, -1, -1, -1]]

lines = cv2.HoughLinesP(image=edges, rho=1, theta=1*np.pi/180, threshold=25, minLineLength=5, maxLineGap=50)

for points in lines:
  x1, y1, x2, y2 = points[0]
  #print("counter is {} line is {}".format(counter, points))
  cv2.line(edges, (x1, y1), (x2,y2), 255, 3)
  cv2.circle(edges, (x1,y1), 10, 255, 2)
  cv2.circle(edges, (x2,y2), 10, 255, 2)

plt.figure(figsize=(8,8))
plt.imshow(edges, cmap='gray')
plt.axis('off')

In [ ]:
smoothOutLines()
newLines = deleteNullLines(lines)
reOrientLines(newLines)
connectVertices(newLines)

background = np.zeros(edges.shape)

for newLine in newLines:
  x1, y1, x2, y2 = newLine[0]
  #print("counter is {} line is {}".format(counter, points))
  cv2.line(background, (x1, y1), (x2,y2), 255, 3)
  cv2.circle(background, (x1,y1), 10, 255, 2)
  cv2.circle(background, (x2,y2), 10, 255, 2)

plt.figure(figsize=(8,8))
plt.imshow(background, cmap='gray')
plt.axis('off')

In [ ]:
segments = findSegments(newLines)
# connectSegments(segments)

area = 0
numHexagons = 0

for segment in segments:
  area, numHexagons = findOuterHexagonArea(segment, area, numHexagons)

for segment in segments:
  segment.printToPicture(edges)
#segment.printAng()

# for line in newLines:
#   x1, y1, x2, y2 = line[0]
#   print("x1 {} y1 {} x2 {} y2 {}".format(x1, y1, x2, y2))
averageArea = area/numHexagons
estimation = np.sum(closed_image) / (averageArea * 255)

print(estimation)

In [ ]:
results_list = []

# loop through each blob:
for blob_number in range(1, np.max(labeled)+1):
  # make sure the blob size is bigger than a threshold. Else the blob is too small and noisy
  if np.sum(np.where(labeled==blob_number, 1, 0))>cutoff_area:
    # get an image with just the current blob
    filtered_image = np.where(labeled==blob_number, 1, 0)
    filtered_image = filtered_image.astype('uint8')
    # copy that image
    res = filtered_image.copy()
    # fill any holes in that image
    contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
      cv2.drawContours(res,[cnt],0,255,-1)
    # copy the closed image
    closed_image = res.copy()

    ### ---- YOUR CODE HERE ---- 

    edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

    width = len(edges[0])
    height = len(edges)
    ANG = pi/12
    NullLine = [[-1, -1, -1, -1]]

    lines = cv2.HoughLinesP(image=edges, rho=1, theta=1*np.pi/180, threshold=25, minLineLength=5, maxLineGap=50)
    smoothOutLines()
    newLines = deleteNullLines(lines)
    reOrientLines(newLines)
    connectVertices(newLines)

    segments = findSegments(newLines)
    # connectSegments(segments)

    area = 0
    numHexagons = 0

    for segment in segments:
      area, numHexagons = findOuterHexagonArea(segment, area, numHexagons)

    for segment in segments:
      segment.printToPicture(edges)
#segment.printAng()

# for line in newLines:
#   x1, y1, x2, y2 = line[0]
#   print("x1 {} y1 {} x2 {} y2 {}".format(x1, y1, x2, y2))
    averageArea = area/numHexagons
    estimation = np.sum(closed_image) / (averageArea * 255)


    ### ---- YOUR CODE HERE ---- 

    # append the analysed blob image to the results list
    results_list.append({'name':'blob number {0}; {1} hexagons'.format(blob_number, round(estimation, 3)), # change variable name with whatever your estimation value variable is
                          'array':edges, # Change `edges` with whatever your output variable name is
                          'areal_value': estimation, # change variable name with whatever your estimation value variable is
                          'given_value': int(round(estimation, 0)), # change variable name with whatever your estimation value variable is
                          'image': image_number+1,
                          'blob': blob_number})
      
# results_list

res = [ sub['name'] for sub in results_list ]
print(res)

In [ ]:
print(areal_value)

# Analysis of true data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
ls ../data/masks

In [ ]:
# Step 1: Get the filepaths of the frames

# set name of folder with all the images ----> ENTER PATH HERE
# dir_name = '/content/drive/MyDrive/Gr_images_Set_4'
dir_name = 'combined_data'

# check if the directory exists; if yes, make a list of all the filepaths
if os.path.isdir(dir_name):
  frame_paths = []
  for dirpath, dirnames, filenames in os.walk(dir_name):
    for name in filenames:
      if 'mask' in name:
        frame_paths.append(os.path.join(dirpath, name))
else:
  raise NameError('')

In [ ]:
frame_paths.sort()
frame_paths

In [ ]:
# allImages = [1, 4, 10, 11, 12, 22, 23] # add your image numbers here

# image_paths = []
# mask_paths_other = []
# mask_paths_testing = []
# for path in frame_paths:
#   if 'mask' in path:
#     if path.split('_')[-1][:-4] in [str(i) for i in allImages]:
#       mask_paths_testing.append(path)
#     else:
#       mask_paths_other.append(path)
#   else:
#     image_paths.append(path)

In [ ]:
len(frame_paths)

In [ ]:
mask_paths_testing = frame_paths[:]

In [ ]:
# results_list[2]['name']
# float(results_list[2]['name'].split()[5])
mask_paths_testing

In [ ]:

kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)
structure = np.ones((3, 3), dtype=int)
results_list = []

filter_area = 5000 # change as per requirement

for p in range(len(mask_paths_testing)):
  path = mask_paths_testing[p]
  print('index: ', p, '; path: ', path)
  
  image = cv2.imread(path, 0)

  erosion = cv2.erode(image, kernel, iterations=2)
  dilation = cv2.dilate(erosion, kernel, iterations=2)

  labeled, ncomponents = label(dilation, structure)

  results_list.append({'name':path.split('/')[-1],
                       'array':image})

  for blob_number in range(1, np.max(labeled)+1):
    try:
      if np.sum(np.where(labeled==blob_number, 1, 0))>filter_area:
        filtered_image = np.where(labeled==blob_number, 1, 0)
        filtered_image = filtered_image.astype('uint8')

        res = filtered_image.copy()
        contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contour:
          cv2.drawContours(res,[cnt],0,255,-1)

        closed_image = res.copy()

        ### ---- YOUR CODE HERE ---- 

        edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

        width = len(edges[0])
        height = len(edges)
        ANG = pi/12
        NullLine = [[-1, -1, -1, -1]]

        lines = cv2.HoughLinesP(image=edges, rho=1, theta=1*np.pi/180, threshold=13, minLineLength=15, maxLineGap=20)

        smoothOutLines()
        newLines = deleteNullLines(lines)
        reOrientLines(newLines)
        connectVertices(newLines)

        segments = findSegments(newLines)
        # connectSegments(segments)

        area = 0
        numHexagons = 0

        for segment in segments:
          area, numHexagons = findOuterHexagonArea(segment, area, numHexagons)

        #estimation = area/numHexagons
        #print("area {} numHexagons {}".format(area, numHexagons))
        if numHexagons > 0:
          averageArea = area / numHexagons
          estimation = np.sum(closed_image) / (averageArea * 255)
          #print("THE ESTIMATED NUMBER OF HEXAGONS IS {}".format(estimation))
        else:
          estimation = 0
          #print("NO HEXAGONS DETECTED")

        for segment in segments:
          segment.printToPicture(edges)
          # print("break")
          # segment.listprint()

        ### ---- YOUR CODE HERE ---- 

        results_list.append({'name':'blob number {0}; {1} hexagons; {2} avg area; {3} area'.format(blob_number, estimation, averageArea, np.sum(closed_image)/255), # change variable name with whatever your estimation value variable is
                            'array':edges}) # Change `lines` with whatever your output variable name is
    except:
      pass
    # else:
    #   filtered_image = np.where(labeled==blob_number, 1, 0)
    #   filtered_image = filtered_image.astype('uint8')

    #   res = filtered_image.copy()
    #   contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

    #   for cnt in contour:
    #     cv2.drawContours(res,[cnt],0,255,-1)

    #   closed_image = res.copy()
    #   results_list.append({'name':'blob number {0}; {1} hexagons'.format(blob_number, 1.00), # change variable name with whatever your estimation value variable is
    #                        'array':closed_image}) # Change `lines` with whatever your output variable name is

numberOfImages = len(results_list)

columns = 3 # you can change this based on what feels most comfortable for your viewing
fig, axes = plt.subplots(numberOfImages//columns+1, columns, figsize=(12, 8*(numberOfImages//columns+1)/columns), sharex=False, sharey=False)
for i, ax in enumerate(axes.flatten()):
  if i<numberOfImages:
    ax.imshow(results_list[i]['array'])
    ax.title.set_text(results_list[i]['name'].split()[:5])
plt.tight_layout()


In [ ]:
len(results_list)

In [ ]:
myarr = []
myarr.append(['name', 0])
myarr[0][1] += 1
myarr

In [ ]:
hexagon_count_array = []
image_count = -1
for i in range(len(results_list)):
  if 'mask' in results_list[i]['name']:
    hexagon_count_array.append([results_list[i]['name'], 0, 0, 0, 0])
    image_count += 1
  else:
    hexagon_count_array[image_count][1] += float(results_list[i]['name'].split()[3])
    hexagon_count_array[image_count][2] += float(results_list[i]['name'].split()[5])
    hexagon_count_array[image_count][3] += float(results_list[i]['name'].split()[8])
    hexagon_count_array[image_count][4] += 1

In [ ]:
len(hexagon_count_array) # [filename, number of hex, total of avg areas, total area, number of blobs]

In [ ]:
df_hexagons = pd.DataFrame(hexagon_count_array, columns=['filename', 'number of hex', 'total of avg areas', 'total area', 'number of blobs'])
df_hexagons

In [ ]:
df_hexagons.to_csv('hexagon_semi_processed_data.csv')

----- BREAK -----

In [ ]:
df_hex_raw = pd.read_csv('hexagon_semi_processed_data_set6.csv', index_col=0)
df_hex_raw

In [ ]:
kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)
structure = np.ones((3, 3), dtype=int)

df_hex_raw["True Hex Num"] = np.zeros(df_hex_raw.shape[0])
df_hex_raw["avg domain size"] = np.zeros(df_hex_raw.shape[0])
for index in range(df_hex_raw.shape[0]):
  if df_hex_raw.iloc[index,0]=='poor':
    df_hex_raw.iloc[index,7] = np.nan
    df_hex_raw.iloc[index,8] = np.nan
  elif df_hex_raw.iloc[index,0]=='good':
    df_hex_raw.iloc[index,7]=(df_hex_raw.iloc[index,2]+df_hex_raw.iloc[index,6])/2
    df_hex_raw.iloc[index,8] = df_hex_raw.iloc[index,4]/df_hex_raw.iloc[index,7]
  elif df_hex_raw.iloc[index,0]=='blobs':
    path = 'combined_data/' + df_hex_raw.iloc[index,1]
    print(path)
    image = cv2.imread(path, 0)
    erosion = cv2.erode(image, kernel, iterations=2)
    dilation = cv2.dilate(erosion, kernel, iterations=2)
    labeled, ncomponents = label(dilation, structure)

    blob_areas = np.zeros((ncomponents, 2))
    for i in range(ncomponents):
      blob_areas[i] = ([i, np.sum(np.where(labeled==i, 1, 0))])
    y = blob_areas[1:, 1]
    y_sort = np.sort(y)
    x = range(1, y_sort.shape[0]+1)

    kn = KneeLocator(x, y_sort, curve='convex', direction='increasing')
    area = y_sort[kn.knee-1]
    df_hex_raw.iloc[index,7] = np.nan
    df_hex_raw.iloc[index,8] = area/2
  else:
    df_hex_raw.iloc[index,7] = float(df_hex_raw.iloc[index,0])
    df_hex_raw.iloc[index,8] = df_hex_raw.iloc[index,4]/df_hex_raw.iloc[index,7]

In [ ]:
# np.unique(df_hex_raw.iloc[:, 0])
df_hex_raw

In [ ]:
df_hex_raw.to_csv('hexagon_fully_processed_data.csv')

------ BREAK -------

In [ ]:
df_hex_full = pd.read_csv('hexagon_fully_processed_data_set6.csv', index_col=0)
df_hex_full

In [ ]:
name_held = '20200922_1'
edge_length = 0
count = 1
for index in range(df_hex_full.shape[0]):
  # print(df_hex_full.iloc[index, 1].split("_")[0]+"_"+df_hex_full.iloc[index, 1].split("_")[1])
  if df_hex_full.iloc[index, 1].split("_")[0]+"_"+df_hex_full.iloc[index, 1].split("_")[1] != name_held:
    print('sample: ', name_held)
    try:
      print('mean domain size: ', edge_length/count)
    except:
      pass
    
    name_held = df_hex_full.iloc[index, 1].split("_")[0]+"_"+df_hex_full.iloc[index, 1].split("_")[1]
    count = 0
    edge_length = 0
  if df_hex_full.iloc[index, 0]!='blobs':
    if df_hex_full.iloc[index, 10] == 'good':
      edge_length += df_hex_full.iloc[index, 9]*(1000/int(df_hex_full.iloc[index, 1].split("_")[3]))
      count += 1
  elif df_hex_full.iloc[index, 0]=='blobs':
    if df_hex_full.iloc[index, 10]=='good':
      edge_length += df_hex_full.iloc[index, 9]*(1000/int(df_hex_full.iloc[index, 1].split("_")[3]))
      count += 1
    elif df_hex_full.iloc[index, 10]=='yes':
      edge_length += df_hex_full.iloc[index, 9]*(1000/int(df_hex_full.iloc[index, 1].split("_")[3]))/2
      count += 1

In [ ]:
print('sample: ', name_held)
print('mean domain size: ', edge_length/count)

# Experimental section: Code blocks used to trace out code in case of problems

*NOTE: @James, please remove any unnecessary cells and reorder the rest.*

## James

In [ ]:
#closed_image = np.uint8(img[canvas_stretch//2:-canvas_stretch//2, canvas_stretch//2:-canvas_stretch//2].copy())
closed_image = filtered_image
edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

lines = cv2.HoughLinesP(image=edges, rho=1, theta=1*np.pi/180, threshold=13, minLineLength=15, maxLineGap=20)

smoothOutLines()
newLines = deleteNullLines(lines)
reOrientLines(newLines)
connectVertices(newLines)

segments = findSegments(newLines)
      # connectSegments(segments)

area = 0
numHexagons = 0

for segment in segments:
  area, numHexagons = findOuterHexagonArea(segment, area, numHexagons)

for segment in segments:
 segment.printToPicture(edges)
 print("break")
 segment.listprint()
 #segment.printAng()

# for line in newLines:
#   x1, y1, x2, y2 = line[0]
#   print("x1 {} y1 {} x2 {} y2 {}".format(x1, y1, x2, y2))
print(len(segments))
if numHexagons > 0:
  averageArea = area/numHexagons
  estimation = np.sum(closed_image) / (averageArea)
  print("Number of Hexagons is: {}".format(estimation))


plt.imshow(edges)

In [ ]:
frame_paths.index('combined_data/image_mask_1.png')

In [ ]:
x = 4 #0 1 4 7 9 10
img_raw = cv2.imread(frame_paths[x], 0)
print(frame_paths[x])

# img_rgray1

plt.imshow(img_raw, cmap='gray')


## Aagam

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import copy
import math
import shapely
import random
from shapely.geometry import LineString, Point
from sympy import Point, Line, pi, sin
from skimage import util
from scipy.ndimage.measurements import label
from sklearn.cluster import KMeans, MiniBatchKMeans
from PIL import Image, ImageDraw
from matplotlib import cm

In [ ]:
mask_paths_testing = ['20221122_2_F2_500_mask.png']
# img_in = cv2.imread(img_path, 0)
# plt.imshow(img_in, cmap='gray')

kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)
structure = np.ones((3, 3), dtype=int)
results_list = []

# filter_area = 5000 # change as per requirement

for p in range(len(mask_paths_testing)):
  path = mask_paths_testing[p]
  print('index: ', p, '; path: ', path)
  
  image = cv2.imread(path, 0)

  erosion = cv2.erode(image, kernel, iterations=2)
  dilation = cv2.dilate(erosion, kernel, iterations=2)

  labeled, ncomponents = label(dilation, structure)

In [ ]:
plt.imshow(labeled)

In [ ]:
blob_areas = np.zeros((ncomponents, 2))
for i in range(ncomponents):
  blob_areas[i] = ([i, np.sum(np.where(labeled==i, 1, 0))])

In [ ]:
y = blob_areas[1:, 1]
y_sort = np.sort(y)

x = range(1, y_sort.shape[0]+1)

# plt.plot(y_sort)
# n, bins, patches = plt.hist(blob_areas[1:,1], bins=[0,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000])
# n, bins, patches = plt.hist(y, bins=[0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000])
# blob_areas[1:,1]

In [ ]:
from kneed import KneeLocator
kn = KneeLocator(x, y_sort, curve='convex', direction='increasing')
print(kn.knee)
print('Area: ', y_sort[kn.knee-1])
plt.figure(figsize=(8,6))
plt.xlabel('blob number', size=16)
plt.ylabel('Area of blob', size=16)
plt.plot(x, y_sort, 'b-')
plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed', label='Cutoff')
plt.legend(fontsize=12)

In [ ]:
y_filtered = y[(y<y_sort[kn.knee-1])]
y_filtered

In [ ]:
print(np.mean(y_filtered))
# n, bins, patches = plt.hist(y_filtered, bins='auto')

## Edge detection techniques 
*(I've stil kept this section header for your convenience. We do not need it)*

In [ ]:
# image = img_raw

# kernel_size = 3
# kernel = np.ones((kernel_size, kernel_size), np.uint8)

# structure = np.ones((3, 3), dtype=np.int)  # this defines the connection filter

# labeled, ncomponents = label(image, structure)

# blob_number = 1 # you can choose the blob label that you want to use
# filtered_image = np.where(labeled==blob_number, 1, 0)
# filtered_image = filtered_image.astype('uint8')

# print(len(filtered_image[0]))
# print(filtered_image)


# plt.imshow(filtered_image, cmap='gray')
########################################################
image = img_raw

kernel_size = 3
kernel = np.ones((kernel_size, kernel_size), np.uint8)

# dilation = cv2.dilate(image, kernel, iterations=5)
# erosion = cv2.erode(dilation, kernel, iterations=5)

erosion2 = cv2.erode(image, kernel, iterations=2)
dilation2 = cv2.dilate(erosion2, kernel, iterations=2)

structure = np.ones((3, 3), dtype=int)  # this defines the connection filter

labeled, ncomponents = label(dilation2, structure)
plt.imshow(labeled, cmap='gray')

blob_number = 1 # you can choose the blob label that you want to use
filtered_image = np.where(labeled==blob_number, 1, 0)
filtered_image = filtered_image.astype('uint8')

# print(len(filtered_image[0]))
# print(filtered_image)

plt.imshow(filtered_image, cmap='gray')

In [ ]:


# for blob_number in range(0, np.max(labeled) + 1):
#   if np.sum(np.where(labeled==blob_number, 1, 0))>10000:
#     filtered_image = np.where(labeled==blob_number, 1, 0)
#     filtered_image = filtered_image.astype('uint8')
#     plt.imshow(filtered_image, cmap='gray')
#   else:
#     print('Blob too small, pick another one. \nCurrent blob number: {0} \nTotal blobs: {1}'.format(blob_number, np.max(labeled)))

In [ ]:
res = filtered_image.copy()
contour,hier = cv2.findContours(res,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

for cnt in contour:
    cv2.drawContours(res,[cnt],0,255,-1)

closed_image = res.copy()

plt.imshow(closed_image, cmap='gray')

In [ ]:
dilation = cv2.dilate(closed_image, kernel, iterations=12)
erosion = cv2.erode(dilation, kernel, iterations=12)

plt.imshow(erosion, cmap='gray')

In [ ]:
edges = cv2.Canny(closed_image, threshold1=.2, threshold2=.5, apertureSize = 3)

lines = cv2.HoughLinesP(edges, 1, 1*np.pi/180, threshold = 13, minLineLength = 15, maxLineGap = 20) # 25 20 ## 13 20

In [ ]:
blob_number = 1  # you can choose the blob label that you want to use
if np.sum(np.where(labeled==blob_number, 1, 0))>5000:
  filtered_image = np.where(labeled==blob_number, 1, 0)
  filtered_image = filtered_image.astype('uint8')
  plt.imshow(filtered_image, cmap='gray')
else:
  print('Blob too small, pick another one. \nCurrent blob number: {0} \nTotal blobs: {1}'.format(blob_number, np.max(labeled)))

In [ ]:
 def tangleBetweenLines(lineA, lineB):
  xStartA, yStartA, xEndA, yEndA = lineA[0]
  xStartB, yStartB, xEndB, yEndB = lineB[0]
  l1 = Line((xStartA, yStartA), (xEndA, yEndA))
  l2 = Line((xStartB, yStartB), (xEndB, yEndB))
  # xTest, yTest = findMidpoint(xStartA, yStartA, xEndB, yEndB)
  # directionCoef = 1
  # if closed_image[yTest][xTest] > 100:
  #   directionCoef = -1
  directionCoef = -1
  #rad = pi + (directionCoef * l1.angle_between(l2))
  rad = l1.angle_between(l2)
  return rad

A = [[0, 0, -10, 0]]
B = [[-10, 0, -15, 9]]
C = [[-15, 9, -10, 18]]
D = [[-10, 18, 0, 18]]
E = [[0, 18, 5, 9]]
F = [[5, 9, 0, 0]]

rad = tangleBetweenLines(B, C)
print(rad)

In [ ]:
np.unique(edges)
plt.figure(figsize=(14,21))
plt.imshow(edges)

In [ ]:
##fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True, sharey=True)
#ax = axes.ravel()

#ax[0].imshow(image, cmap=cm.gray)
#ax[0].set_title('Input image')

#ax[1].imshow(edges, cmap=cm.gray)
#ax[1].set_title('Canny edges')

#ax[2].imshow(erosion, cmap=cm.gray)
#ax[2].set_title('Denoised edges')

#plt.tight_layout()
#plt.show()

#find Countors Cv


In [ ]:
def traceOutline(i, j):
  outline, erosionCopy[i][j], counter = [[i, j]], 50, 0
  while counter < len(outline):
    [y, x] = outline[counter]
    if x != 0 and y != 0 and erosionCopy[y-1][x-1] > 100:
      outline.append([y-1, x-1])
      erosionCopy[y-1][x-1] = 50
    if y != 0 and erosionCopy[y-1][x] > 100:
      outline.append([y-1, x])
      erosionCopy[y-1][x] = 50
    if x < len(erosion[0]) - 1 and y != 0 and erosionCopy[y-1][x+1] > 100:
      outline.append([y-1, x+1])
      erosionCopy[y-1][x+1] = 50
    if x != 0 and erosionCopy[y][x-1] > 100:
      outline.append([y, x-1])
      erosionCopy[y][x-1] = 50
    if x < len(erosion[0]) - 1 and erosionCopy[y][x+1] > 100:
      outline.append([y, x+1])
      erosionCopy[y][x+1] = 50
    if x != 0 and y < len(erosion) - 1 and erosionCopy[y+1][x-1] > 100:
      outline.append([y+1, x-1])
      erosionCopy[y+1][x-1] = 50
    if y < len(erosion) - 1 and erosionCopy[y+1][x] > 100:
      outline.append([y+1, x])
      erosionCopy[y+1][x] = 50
    if x<len(erosion[0])-1 and y<len(erosion)-1 and erosionCopy[y+1][x+1]>100:
      outline.append([y+1, x+1])
      erosionCopy[y+1][x+1] = 50
    counter += 1

def traceOutlineDepth(i, j):
  outline, stack, erosionCopy[i][j] = [[i,j]], [[i,j]], 50
  while len(stack) > 0:
    [y, x] = stack[len(stack)]
    if x != 0 and y != 0 and erosionCopy[y-1][x-1] > 100:
      outline.append([y-1, x-1])
      erosionCopy[y-1][x-1] = 50
      stack.append([y-1, x-1])
      continue
    if y != 0 and erosionCopy[y-1][x] > 100:
      outline.append([y-1, x])
      erosionCopy[y-1][x] = 50
    if x < len(erosion[0]) - 1 and y != 0 and erosionCopy[y-1][x+1] > 100:
      outline.append([y-1, x+1])
      erosionCopy[y-1][x+1] = 50
    if x != 0 and erosionCopy[y][x-1] > 100:
      outline.append([y, x-1])
      erosionCopy[y][x-1] = 50
    if x < len(erosion[0]) - 1 and erosionCopy[y][x+1] > 100:
      outline.append([y, x+1])
      erosionCopy[y][x+1] = 50
    if x != 0 and y < len(erosion) - 1 and erosionCopy[y+1][x-1] > 100:
      outline.append([y+1, x-1])
      erosionCopy[y+1][x-1] = 50
    if y < len(erosion) - 1 and erosionCopy[y+1][x] > 100:
      outline.append([y+1, x])
      erosionCopy[y+1][x] = 50
    if x<len(erosion[0])-1 and y<len(erosion)-1 and erosionCopy[y+1][x+1]>100:
      outline.append([y+1, x+1])
      erosionCopy[y+1][x+1] = 50

counter = 0
erosionCopy = copy.deepcopy(erosion)
for i in range(0, len(erosion)):
  for j in range(0, 500):
      if erosionCopy[i][j] > 100:
        counter += 1
        traceOutline(i, j)

fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(erosion, cmap=cm.gray)
ax[1].set_title('Erosion edges')

ax[2].imshow(erosionCopy, cmap=cm.gray)
ax[2].set_title('Copy edges')

plt.tight_layout()
plt.show()


